In [1]:
import cv2
import numpy as np
from datetime import datetime
import pytz
from math import *
import os, shutil
import fitz
import re
import PyPDF2
import matplotlib.pyplot as plt

In [2]:
#pdfname="16012023"
pdfname = 'uk'

In [3]:
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:  # 判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)  # makedirs 创建文件时如果路径不存在会创建这个路径
    else:
        pass

'''
print(STACK[1])
print(STACK[1][1])
print(STACK[1][1][5:])
详细解释一下STACK的构成，三级list，
STACK[1] 一级列表，元素为子列表，每个子列表包含一整页邮编的SD
STACK[1][1] 第二级列表，其子元素为某一张邮票SD
STACK[][][] 第三级列表，其子元素是某SD的字符组成
['SD : 87500070634473F', 'SD : 87500070634474D', 'SD : 87500070634475B', 'SD : 875000706344769'.........]
SD : 87500070634474D
87500070634474D
'''
STACK=[]        # 邮票上的SD号码
pdf_header="pdfs/"

def pdf_image(pdfPath, imgPath, zoom_x, zoom_y, rotation_angle):
    """
    inpput: 由la poste下载的邮票总pdf
    output: 将总pdf按页切割,生成图片jpg

    @param pdfPath: pdf文件的路径
    @param imgPath: 图像要保存的文件夹
    @param zoom_x: x方向的缩放系数
    @param zoom_y: y方向的缩放系数
    @param rotation_angle: 旋转角度

    使用过的总pdf移入 used-pdf/文件中
    生成的jpg图片放入images/pdfname/文件中
                                    
    """
    # 打开PDF文件

    pdf = fitz.open(pdfPath)
    pdfReader = PyPDF2.PdfFileReader(pdfPath)
    name = pdf.name
    # 将前面的文件夹路径替换，以及把后缀的地方去掉
    name = name.replace('pdfs/', '').replace('.pdf', '')
    # 逐页读取PDF
    # 有一些小函数在小细节上面做出了更新
    for pg in range(0, pdf.page_count):
        page = pdf.load_page(pg)

        ###########add-stack
        pagehandle = pdfReader.getPage(pg)
        pagestack=re.findall("SD.*",pagehandle.extractText())
        STACK.append(pagestack)
        # 设置缩放和旋转系数
        trans = fitz.Matrix(zoom_x, zoom_y).prerotate(rotation_angle)
        pm = page.get_pixmap(matrix=trans, alpha=False)
        # 开始写图像
        mkdir(imgPath + name)
        ###现在直接用save函数保存图片了、
        pm.save(imgPath + name + '/' + str(pg + 1) + ".png")
    pdf.close()
    shutil.move(pdfPath,"./used-pdf/")


pdf_image(pdf_header+pdfname+'.pdf', r"images/", 5, 5, 0)

In [9]:
##################################切割正常的邮票#####################################

'''
切割图片jpg,等到单张邮票
Input:由上面程序切割得到的图片jpg,每张图片中包含很多张单张邮票
使用过的图片jpg移入used-pdfpage
Output:单张邮票集合,集合存入 sousimgs中

'''

imgfiles="./images/"
file_dir=imgfiles+pdfname+'/'

file_list = []
for items in os.walk(file_dir, topdown=False):
    file_list = items[2]

filelistcount=0
for file in file_list:
    pic_path =file_dir+file # 分割的图片的位置
    pic_target = './sousimgs/' # 分割后的图片保存的文件夹
    if not os.path.exists(pic_target):  #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(pic_target)
   
    cut_width = 520
    cut_length = 992

    # 读取要分割的图片，以及其尺寸等数据
    picture = cv2.imread(pic_path)
    (width, length, depth) = picture.shape
    # 预处理生成0矩阵
    pic = np.zeros((cut_width, cut_length, depth))
    # for循环迭代生成
    timbrecount=0
    for i in range(0, 8):
        for j in range(0, 3):
            pic = picture[i*cut_width : (i+1)*cut_width, j*cut_length : (j+1)*cut_length, :]      
            result=STACK[filelistcount][timbrecount][5:]
            result_path = pic_target + '{}.jpg'.format(result)              # 这一步是用邮票的SD编号对单张邮票进行命名
            cv2.imwrite(result_path, pic)
            timbrecount+=1
    filelistcount+=1
shutil.move(file_dir ,"./used-pdfpage/")
print()

IndexError: list index out of range

In [4]:
################切割 UK 的邮票#####################


imgfiles="./images/"
file_dir=imgfiles+pdfname+'/'

file_list = []
for items in os.walk(file_dir, topdown=False):
    file_list = items[2]

filelistcount=0
for file in file_list:
    pic_path =file_dir+file # 分割的图片的位置
    pic_target = './sousimgs_UK/' # 分割后的图片保存的文件夹
    if not os.path.exists(pic_target):  #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(pic_target)
   
    cut_width = 550
    cut_length = 992

    # 读取要分割的图片，以及其尺寸等数据
    picture = cv2.imread(pic_path)
    (width, length, depth) = picture.shape
    # 预处理生成0矩阵
    pic = np.zeros((cut_width, cut_length, depth))
    # for循环迭代生成
    timbrecount=0
    try:
        for i in range(0, 8):
            for j in range(0, 3):
                pic = picture[i*cut_width : (i+1)*cut_width, j*cut_length : (j+1)*cut_length, :]      
                result=STACK[filelistcount][timbrecount][5:]
                result_path = pic_target + '{}.png'.format(result)              # 这一步是用邮票的SD编号对单张邮票进行命名
                cv2.imwrite(result_path, pic)
                timbrecount+=1
        filelistcount+=1
    except Exception:
        print('excrption for index, but program continue,its ok. ')
shutil.move(file_dir ,"./used-pdfpage/")


excrption for index, but program continue,its ok. 


'./used-pdfpage/uk'